# Chat Completions

Chat models take a series of messages as input, and return a model-generated message as output.
The main input is the messages parameter. **Messages must be an array of message objects, where each object has a role (either "system", "user") and content** (the content of the message). 

In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
#%pip install --upgrade openai

In [3]:
# import openai
# from openai import AzureOpenAI
# import os 
# from azure.identity import ManagedIdentityCredential

# default_credential=ManagedIdentityCredential(client_id="d30cba06-04c1-4065-a91d-8b7ce3b07b78")
# token=default_credential.get_token("https://cognitiveservices.azure.com/.default")
# Resource_endpoint="https://openaiykus.openai.azure.com/"

# client = AzureOpenAI(
#   azure_endpoint = Resource_endpoint, 
#   api_key=token.token,  
#   api_version="2023-05-15"
# )

In [1]:
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_type = "azure"
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",#2024-02-01
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [3]:
messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you write a welcome page for The Very Group company website ?"}]

In [4]:
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=messages
)
print(response.choices[0].message.content)

Welcome to The Very Group, a company that integrates creativity, innovation, and cutting-edge technology to create unique and exceptional customer experiences.

We aim to connect, inspire and transform the way people live, work, and play through our diverse range of products and services. With our comprehensive collection that spans from categories including fashion, home, beauty, tech, sports and more, we are at the forefront of the retail industry.

Our passion for what we do is reflected not only in the quality of our products but also in the exceptional service we provide. As pioneers of online retail and financial services, we combine real-world insights with digital expertise to deliver the best for our customers. 

We are more than just a company - we are a community, a network, and a platform for positive change. We are committed to being a force for good in society, empowering our customers, our people, and our partners to reach their fullest potential.

This is what different

In [5]:
response

ChatCompletion(id='chatcmpl-9AVRVdSA7IQf558S2bFFlQa3dsNA7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Welcome to The Very Group, a company that integrates creativity, innovation, and cutting-edge technology to create unique and exceptional customer experiences.\n\nWe aim to connect, inspire and transform the way people live, work, and play through our diverse range of products and services. With our comprehensive collection that spans from categories including fashion, home, beauty, tech, sports and more, we are at the forefront of the retail industry.\n\nOur passion for what we do is reflected not only in the quality of our products but also in the exceptional service we provide. As pioneers of online retail and financial services, we combine real-world insights with digital expertise to deliver the best for our customers. \n\nWe are more than just a company - we are a community, a network, and a platform for positive change. 

In [6]:
messages.append({"role": "system", "content": response.choices[0].message.content})

In [7]:
messages.append({"role": "user", "content": "Can you summarize our conversation?"})

In [8]:
print(messages)

[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Can you write a welcome page for The Very Group company website ?'}, {'role': 'system', 'content': "Welcome to The Very Group, a company that integrates creativity, innovation, and cutting-edge technology to create unique and exceptional customer experiences.\n\nWe aim to connect, inspire and transform the way people live, work, and play through our diverse range of products and services. With our comprehensive collection that spans from categories including fashion, home, beauty, tech, sports and more, we are at the forefront of the retail industry.\n\nOur passion for what we do is reflected not only in the quality of our products but also in the exceptional service we provide. As pioneers of online retail and financial services, we combine real-world insights with digital expertise to deliver the best for our customers. \n\nWe are more than just a company - we are a community, a network, and 

In [9]:
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=messages
)
print(response.choices[0].message.content)

You requested that I write a welcome page for The Very Group's company website. I composed a welcoming message that describes the company as an innovative and creative force in the retail industry. The summary touches on their comprehensive product range, commitment to exceptional customer service, and dedication to empowering customers, employees, and partners. Furthermore, the message encourages visitors to explore the website and welcomes them to a world of endless opportunities.


In [10]:
# Instead of appending, writing messages in the SDK
response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you write a welcome page for The Very Group company website ?"},
        {"role": "system", "content": "Welcome to The Very Group, a company that integrates creativity, innovation, and cutting-edge technology to create unique and exceptional customer experiences."},
        {"role": "user", "content": "Can you summarize our conversation?"},
    ]
)

print(response.choices[0].message.content)

You asked me to write a welcome page for The Very Group's company website, which I did by encapsulating their essence as a creative, innovative, and technology-driven company dedicated to providing unique customer experiences.


# PII Redaction

In [11]:
content= f"""You are a helpful assistant that answers the given question. You must present both response and original question in your output after redacting any PII data such as name, address, date of birth, medical records, and financial records.
 
Format the output in JSON by using "Original Question" and "Response" as a key. The original question should be the user input verbatim."""

In [12]:

response = client.chat.completions.create(
    model="gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": content},
        {"role": "user", "content": "Hi, I am Yeliz, based in Cambridge. Can you write a poem for me?"}
    ]
)

print(response.choices[0].message.content)

{
  "Original Question": "Hi, I am [redacted], based in [redacted]. Can you write a poem for me?",
  "Response": "Of course! Here is a poem for you:\n\nIn a city of dreams, both old and new,\nWhere knowledge blends with shades of blue,\nThere dwells a soul, both bright and free,\nIn love with life's rich tapestry.\nBeneath the spreading chestnut tree,\nCreating stories, wild and free,\nWith laughter light and spirit high,\nUnderneath the Cambridge sky."
}


### **GPT-4 TURBO  Reproducable Outputs with Seed Parameter**

In [19]:
for i in range(3):
  print(f'Story Version {i + 1}\n---')
    
  response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # Model = should match the deployment name you chose for your 1106-preview model deployment
    #seed=42,
    temperature=0.5,
    max_tokens =50, 
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Tell me a story about how the universe began?"}
    ]
  )
  
  print(response.choices[0].message.content)
  print("---\n")


Story Version 1
---
In the vast tapestry of existence, the story of how the universe began is a tale that has fascinated humans for millennia. Among the many narratives, one of the most compelling is the scientific account, known as the Big Bang theory. This is a
---

Story Version 2
---
In the vast expanse of imagination and science, there are many stories about how the universe began. One of the most captivating and widely accepted by scientists is the tale of the Big Bang.

Once upon a time, about 13.8 billion years
---

Story Version 3
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the birth of everything we know—the universe began with a monumental explosion known as the Big Bang. This isn't just any story;
---



In [20]:
for i in range(3):
  print(f'Story Version {i + 1}\n---')
    
  response = client.chat.completions.create(
    model="gpt-4-0125-Preview", # Model = should match the deployment name you chose for your 1106-preview model deployment
    seed=42,
    temperature=0.5,
    max_tokens =50, 
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "Tell me a story about how the universe began?"}
    ]
  )
  
  print(response.choices[0].message.content)
  print("---\n")


Story Version 1
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the creation of everything we know—the universe's beginning. This is a tale not of science or facts, but of imagination, inspired
---

Story Version 2
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the creation of everything we know—the universe began with the Big Bang.

Before this moment, all that would become the universe was condensed
---

Story Version 3
---
In the vast expanse of nothingness, where time and space had yet to find their form, a singular event sparked the creation of everything we know—the universe's beginning. This is a tale not of science or fact, but of imagination, inspired
---

